# Guided Project 1:  Profitable App Profiles for the App Store and Google Play Market

Our main goal to this project is to determine how we may determine the most profitable apps for Google Play and Apple Store. Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.

In [1]:
def open_files(file):
    open_file = open(file,encoding='utf8')
    from csv import reader
    reader_file = reader(open_file)
    result = list(reader_file)
    return result

In [2]:
def explore_data(dataset, start , end, rows_and_columns =False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
ios_apps = open_files("AppleStore.csv")
android_apps = open_files("googleplaystore.csv")
print(ios_apps[0])
print("\n")
explore_data(ios_apps[1:], 0 , 5, True)
print('\n\n\n')
print(android_apps[0])
print('\n')
explore_data(android_apps[1:], 0 , 5, True)

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']


['4', '282614216', 'eBay: Best App to Buy, Sell, Save! Online Shopping', '128512000', 'USD', '0', '262241', '649', '4', '4.5', '5.10.0', '12+', 'Shopping', '37', '5', '9', '1']


['5', '282935706', 'Bible', '92774400', 'USD', '0', '985920', '5320', '4.5', '5', '7.5.1', '4+', 'Reference', '37'

For Googleplaystore, we see that it has 10841 rows and 13 columns. The interested columns are `App`, `Category`, `Rating`, `Price`, `Type`, `Genres` and `Installes`. As we can see, that some columns aren't descriptive enough, so at this link you may see the [documentation](https://www.kaggle.com/datasets/lava18/google-play-store-apps).

For Applestore, we see that it has 7197 rows and 17 columns. The interested columns are `track name`, `price`, `user_rating`, `prime genre`, `rating_count_tot`. As we can see, that some columns aren't descriptive enough, so at this link you may see the [documentation](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps).

## Deleting Wrong Data

As we read in comentars, the 10473 row have an empty element, so we need delete it:

In [4]:
print(android_apps[10473])
print("\n")
print(len(android_apps))
del(android_apps[10473])
print("\n")
print(len(android_apps))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


10842


10841


## Removing Duplicate Entries

### Part One

_If you explore the Google Play data set long enough or look at the discussions section, you'll notice some apps have duplicate entries. For instance, Instagram has four entries_:

In [5]:
for app in android_apps:
    if app[0] == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In total, there are 1,181 cases where an app occurs more than once:

In [6]:
duplicate_apps = []
unique_apps = []
for app in android_apps[1:]:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('Number of duplicate apps is:', len(duplicate_apps))
print('\n')
print('Some examples of duplicate apps: ', duplicate_apps[:20])

Number of duplicate apps is: 1181


Some examples of duplicate apps:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software', 'MailChimp - Email, Marketing Automation', 'Crew - Free Messaging and Scheduling', 'Asana: organize team projects', 'Google Analytics', 'AdWords Express']


---
---
*We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the last duplicate rows and remain only the first unique, but we could probably find a better way. If we examine the rows we printed for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. So we want to colect the more recent data, so we need to remain the row with highest number of reviews.*

### Part two

Let's start with creating dictionary, in which every key is a app name and each dictionary value is a highest number of reviews of that app:

In [7]:
reviews_max = {}
for app in android_apps[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if (name in reviews_max) and (reviews_max[name] < n_reviews):
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews 

Let's check that we have done everything in right way:

In [8]:
print("Expected length: ", len(android_apps[1:]) - 1181)
print("We have received: ", len(reviews_max))

Expected length:  9659
We have received:  9659


So let's remove the duplicate apps from the dataset:

In [9]:
android_clean = []
already_added = []
for app in android_apps[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if reviews_max[name] == n_reviews and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
print(len(android_clean))

9659


In the cell above we do the following:
* Create an empty lists `android_clean` and `already_added`
* Loop through the dataset `android_apps`:
    * Excract the name of the app(element with index number 0) and save it to a variable `name`
    * Excract the number of reviews(element with index number 3) and save it to a variable `n_reviews`
    * We add the current row (`app`) to the `android_clean` list, and the app name (`name`) to the `already_added` list if `n_reviews` is the same as the number of maximum reviews of the app name (the number can be found in the reviews_max dictionary) and `name` is not already in the list `already_added`.

Let's explore the new cleaning dataset:

In [10]:
explore_data(android_clean, 0, 5 , True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9659
Number of columns: 13


## Removing Non-English apps

### Part one

If we explore the data long enough, we'll find that both datasets have apps with names that suggest they are not designed for an English-speaking audience. So we need to create functions, that explore the string and says, if it has any non-English characters:



In [11]:
def explore_string(string):
    for element in string:
        if(ord(element)) > 127:
            return False;
    return True;

Let's check our solution:

In [12]:
print(explore_string("Instagram"))
print(explore_string('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(explore_string("Docs To Go™ Free Office Suite"))
print(explore_string("Instachat 😜")) 

True
False
False
False


### Part Two

To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range:

In [13]:
def explore_string(string):
    value = 0
    for element in string:
        if ord(element) > 127:
            value += 1
    if value > 3:
        return False
    return True
print(explore_string("Docs To Go™ Free Office Suite"))
print(explore_string("Instachat 😜"))
print(explore_string("爱奇艺PPS -《欢乐颂2》电视剧热播"))

True
True
False


In [14]:
def explore_english(dataset, value):
    result = []
    for row in dataset:
        name = row[value]
        if explore_string(name):
            result.append(row)
    return result

Let's use the function above to clean both datasets and then explore it:

In [15]:
android_english = explore_english(android_clean, 0)
apple_english = explore_english(ios_apps[1:], 2)
explore_data(android_english, 0 , 3, True)
print("\n")
explore_data(apple_english, 0 , 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188

We can see that we're left with 9614 Android apps and 6183 iOS apps.

## Isolating the Free Apps

Our datasets contain both free and non-free apps, we'll need to isolate only the free apps for our analysis:

In [16]:
def excract_free(dataset, value):
    result = []
    for row in dataset:
        price = row[value]
        if price == '0':
            result.append(row)
    return result

Let's see the datasets, that remains after isolate:

In [17]:
android_free = excract_free(android_english, 7)
ios_free = excract_free(apple_english, 5)
print(len(android_free))
print(len(ios_free))

8864
3222


We're left with 8864 Android apps and 3222 iOS apps, which should be enough for our analysis.

## Most Common Apps by Genre

### Part One

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful in both markets.

To minimize risks and overhead, our validation strategy for an app idea has three steps:1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

As we see, we may use the columns `Genres` , `Category` in `GooglePlaystore` dataset, and the column `Prime_genre` in `Applestore` dataset to generate frequency tables to determine the most common genres in each market.ore.

### Part Two

In [18]:
def freq_table(dataset, index):
    length = len(dataset)
    result = {}
    for row in dataset:
        element = row[index]
        if element in result:
            result[element] += 1
        elif element not in result:
            result[element] = 1
    table_percentage = {}
    for key in result:
        percentage = (result[key]/ length) * 100
        table_percentage[key] = percentage
    return table_percentage

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_displays = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_displays.append(key_val_as_tuple)
    table_sorted = sorted(table_displays, reverse = True)
    for entry in table_sorted:
        print(entry[1], ":", entry[0])

### Part three

Now let's display frequency table for column `Prime_Genre` in `AppleStore` dataset

In [19]:
display_table(ios_free, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


We can see, that more than half of all English apps is game apps(58.16%). On the second place of most popular genre is Entertainment apps. As we can see , these two genres occupies approximately 65% of all apps in AppleStore and it's really incredible. 

The general impression of AppleStore(based on free apps) is that most apps are designed for fun and enertainment(games, photo and video, social networking, sports, music), while apps designed for ractical purposes (education, shopping, utilities, productivity, lifestyle) are more rare. However, it doesn't mean, that entertainment apps have a large number of users, because this property depends on other factors.



Let's excract the columns `Category` and `Genres` from `GooglePlay` dataset

In [20]:
display_table(android_free, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

We can see, that the most popular categories are `Family`, `Games` and `Tools`. The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). 

Let's create a frequancy table for `Genres` column

In [21]:
display_table(android_free, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

We can see that columns `Genres` and `Category` are mostly similar, but the column`Genres` describe the different genre of apps more detaily.

The difference between the `AppleStore` dataset from `GooglePlay` dataset is that `AppleStore` have mostly apps for fun and entertainment, while `GooglePlay` have a balanced landscape, that shows balance betweeen apps for fun and apps for practical purposes. But this frequancy tables didn't help us with our main goal, because they don't reveal the most frequent app genres or what genres have the most users.

##  Most Popular Apps by Genre on the App Store

Let's defined the most popular apps by genre on the App Store:

In [22]:
genres_apple = freq_table(ios_free, -5)
for element in genres_apple:
    total = 0
    len_genre = 0
    for row in ios_free:
        genre_app = row[-5]
        if genre_app == element:
            user_ratings = float(row[6])
            total += user_ratings
            len_genre += 1
    avg_rating = total / len_genre
    print(element, ':', avg_rating)

Productivity : 21028.410714285714
Weather : 52279.892857142855
Shopping : 26919.690476190477
Reference : 74942.11111111111
Finance : 31467.944444444445
Music : 57326.530303030304
Utilities : 18684.456790123455
Travel : 28243.8
Social Networking : 71548.34905660378
Sports : 23008.898550724636
Health & Fitness : 23298.015384615384
Games : 22788.6696905016
Food & Drink : 33333.92307692308
News : 21248.023255813954
Book : 39758.5
Photo & Video : 28441.54375
Entertainment : 14029.830708661417
Business : 7491.117647058823
Lifestyle : 16485.764705882353
Education : 7003.983050847458
Navigation : 86090.33333333333
Medical : 612.0
Catalogs : 4004.0


As we can see, that many users used the genre `Navigation`, `Reference` and `Music`. It's really strange, that only 22788 users in average used the genre `Games`, while the percentage of Game apps in `App store` is more than 58%. So we may understand that not always the most popular apps in dataset is the most profitable, because this area has a lot of competition and not the largest number of users. As we can see below the `Navigation` genre has occupies by largest companies, as Google and Waze. So we can expect, that it will be difficult to to displace them from their leadership position. 

In [23]:
for row in ios_free:
    if row[-5] == "Navigation":
        print(row[2], ":", row[6])
        

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Geocaching® : 12811
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5
CoPilot GPS – Car Navigation & Offline Maps : 3582
Google Maps - Navigation & Transit : 154911


So let's display next "large" genre `Reference`:

In [24]:
for row in ios_free:
    if row[-5] == "Reference":
        print(row[2], ":", row[6])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
Merriam-Webster Dictionary : 16849
Google Translate : 26786
Night Sky : 12122
WWDC : 762
Jishokun-Japanese English Dictionary & Translator : 0
教えて!goo : 0
VPN Express : 14
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Real Bike Traffic Rider Virtual Reality Glasses : 8


We can see the same character, but here the picture looks better because there are already more competing applications, and we also see that the number of users in this area is more than 1.5 million. So the app in this genre may be  profitable, because this genre has a lot of users and there is not many existing apps in this sphere, so it hasn't a lot of competition. We can create something app, which will be have a lot of books, and also, may be have chat with other readers to discuss different books. In this app people will be spend a lot of time reading books, discussing it, and also we may create some extra features, which will be people may to buy by some activities in the app. So we can say, that this app will be successfull. Let's go to another genres and analyze it.

Let's display next genres `Music` and `Weather`:

In [25]:
for row in ios_free:
    if row[-5] == "Music":
        print(row[2], ":", row[6])

print("\n")
for row in ios_free:
    if row[-5] == "Weather":
        print(row[2], ":", row[6])

Pandora - Music & Radio : 1126879
Shazam - Discover music, artists, videos & lyrics : 402925
iHeartRadio – Free Music & Radio Stations : 293228
Deezer - Listen to your Favorite Music & Playlists : 4677
Sonos Controller : 48905
NRJ Radio : 38
radio.de - Der Radioplayer : 64
Spotify Music : 878563
SoundCloud - Music & Audio : 135744
Sing Karaoke Songs Unlimited with StarMaker : 26227
SoundHound Song Search & Music Player : 82602
Ringtones for iPhone & Ringtone Maker : 25403
Coach Guitar - Lessons & Easy Tabs For Beginners : 2416
TuneIn Radio - MLB NBA Audiobooks Podcasts Music : 110420
Magic Piano by Smule : 131695
QQ音乐HD : 224
The Singing Machine Mobile Karaoke App : 130
Bandsintown Concerts : 30845
PetitLyrics : 0
edjing Mix:DJ turntable to remix and scratch music : 13580
Smule Sing! : 119316
Amazon Music : 106235
AutoRap by Smule : 18202
My Mixtapez Music : 26286
Certified Mixtapes - Hip Hop Albums & Mixtapes : 9975
Karaoke - Sing Karaoke, Unlimited Songs! : 28606
Napster - Top Music 

We can see, that both genres have a lot of competition. `Music` genre have already a lot of successfull apps, as `Spotify`, `Shazam`, `SoundCloud`, `Pandora`, etc. The `Weather` genre has a lot of disadvantages, one of it is that  eople generally don't spend too much time in-app, and the chances of making profit from in-app adds are low.

*So we can consider that the most profitable app in `App Store` will be the **Book service** in genre `Refference`.*

## Most Popular Apps by Genre on Google Play

In [26]:
display_table(android_free, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

In [27]:
categories_android = freq_table(android_free, 1)


for category in categories_android:
    total = 0
    len_category = 0
    for row in android_free:
        category_app = row[1]
        if category_app == category:
            number_string = row[5]
            number_string = number_string.replace(",", "")
            number_string = number_string.replace("+", "")
            total += float(number_string)
            len_category += 1
    avg_rating = total / len_category
    print(category, ":", avg_rating)
            

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

We can see that the most installs has `NEWS_AND_MAGAZINES` , `VIDEO_PLAYERS`, `PRODUCTIVITY`, `TOOLS`, `COMMUNICATION`, `ENTERTAINMENT`. We also see that the genre we choosed in previous step( `Reference` ) `Books and Reference` has 8M+ installs. It's a good number, because in our time a lot of people don't read books, but the number of readers is still huge. But let's see the most popular category in `GoogleStore` and explain they are not profitable or not.

Let's display the `Communication` category, but we have take only apps, which has more than 5M installs:

In [28]:
for row in android_free:
    if row[1] == "COMMUNICATION" and(row[-8] == "100,000,000+" or row[-8] == "500,000,000+" or row[-8] == "1,000,000,000+" 
                                     or row[-8] == "5,000,000+" or row[-8] == "10,000,000+" or row[-8] == "50,000,000+"):
        print(row[0], ":", row[-8])
        

WhatsApp Messenger : 1,000,000,000+
Messenger for SMS : 10,000,000+
My Tele2 : 5,000,000+
imo beta free calls and text : 100,000,000+
Contacts : 50,000,000+
Call Free – Free Call : 5,000,000+
Web Browser & Explorer : 5,000,000+
Browser 4G : 10,000,000+
MegaFon Dashboard : 10,000,000+
ZenUI Dialer & Contacts : 10,000,000+
Cricket Visual Voicemail : 10,000,000+
Xperia Link™ : 10,000,000+
TouchPal Keyboard - Fun Emoji & Android Keyboard : 10,000,000+
Skype Lite - Free Video Call & Chat : 5,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
AT&T Visual Voicemail : 10,000,000+
GMX Mail : 10,000,000+
Omlet Chat : 10,000,000+
My Vodacom SA : 5,000,000+
Microsoft Edge : 5,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Calls & Text by Mo+ : 5,000,000+
free video calls and chat : 50,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Them

We can see that there is a lot of apps, which we use in our daily life: `Microsoft Edge`, `Firefox`, `Skype`, `Viber`, `Gmail`, `Whats app` and other. Since there are so many well-known apps in this sphere and a lot of competition, we won't choose this category for the our company, because it's will be expensive to start and improve the app in this sphere.

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Let's display the `Games` category, but we have take only apps, which has more than 5M installs:

In [29]:
for row in android_free:
    if row[1] == "GAME" and(row[-8] == "100,000,000+" or row[-8] == "500,000,000+" or row[-8] == "1,000,000,000+" 
                                     or row[-8] == "5,000,000+" or row[-8] == "10,000,000+" or row[-8] == "50,000,000+"):
        print(row[0], ":", row[-8])

Solitaire : 10,000,000+
Sonic Dash : 100,000,000+
PAC-MAN : 100,000,000+
Bubble Witch 3 Saga : 50,000,000+
Race the Traffic Moto : 10,000,000+
Marble - Temple Quest : 10,000,000+
Shooting King : 10,000,000+
Geometry Dash World : 10,000,000+
Jungle Marble Blast : 5,000,000+
Roll the Ball® - slide puzzle : 100,000,000+
Block Craft 3D: Building Simulator Games For Free : 50,000,000+
Love Balls : 50,000,000+
Piano Tiles 2™ : 100,000,000+
Pokémon GO : 100,000,000+
Paint Hit : 10,000,000+
Snake VS Block : 50,000,000+
Rolly Vortex : 10,000,000+
Stack Jump : 10,000,000+
The Cube : 5,000,000+
Extreme Car Driving Simulator : 100,000,000+
Color Road : 10,000,000+
Draw In : 10,000,000+
Trivia Crack : 100,000,000+
Will it Crush? : 5,000,000+
Tomb of the Mask : 5,000,000+
Baseball Boy! : 10,000,000+
Hello Stars : 10,000,000+
Tank Stars : 10,000,000+
Hole.io : 10,000,000+
Mini Golf King - Multiplayer Game : 5,000,000+
Flip the Gun - Simulator Game : 10,000,000+
Word Link : 10,000,000+
Last Day on Ear

We can see that there is a lot of apps, which have a lot of installs. Since there are so many well-known apps in this sphere and a lot of competition, we won't choose this category for the our company, because it's will be expensive to start and improve the app in this sphere.

So let's explore more the Category `BOOKS AND REFERENCE`, because we choose this category for our app in GoogleStore.

In [30]:
for app in android_free:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

We can see that there is not a lot apps in this category, but let's see the most popular apps in this category.

In [31]:
for row in android_free:
    if row[1] == "BOOKS_AND_REFERENCE" and(row[-8] == "100,000,000+" or row[-8] == "500,000,000+" or row[-8] == "1,000,000,000+" 
                                     or row[-8] == "5,000,000+" or row[-8] == "10,000,000+" or row[-8] == "50,000,000+"):
        print(row[0], ":", row[-8])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Aldiko Book Reader : 10,000,000+
Wattpad 📖 Free Books : 100,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Quran for Android : 10,000,000+
Audiobooks from Audible : 100,000,000+
Dictionary.com: Find Definitions for English Words : 10,000,000+
English Dictionary - Offline : 10,000,000+
Bible KJV : 5,000,000+
NOOK: Read eBooks & Magazines : 10,000,000+
Dictionary : 10,000,000+
Spanish English Translator : 10,000,000+
Dictionary - Merriam-

We can see that there is dictinaries in this list, and they don't create to our app any competition, because our app is about reading books and discussing it. We also can see, that most popular apps have only one choice: you may read book and in other app you may hear another, and if you read foreign books it will be appeared, that you want to translate the text of the sentences. But it is inconvenient and people spend a lot of time on that thing. So now we have a model of requirements, which must have our app.

**So if we create multifunctional app, with translator and audiobooks, and in this app people will be discuss the read books, we will get a powerfull app, which is profitable for AppleStore and GooglePlay.**

## Conclusion

In this project, we analyzed the data from two datasets: App Store and Google Play Market with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a lot of books and transform it in the app will be very effective for both the Google Play and the App Store markets. . But we also concluded, that this app will be very similar to other apps. So we need to add some special features besides to this app. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.